<img src="src/xtensor.png" />

The lazy Tensor Algebra Expression Library

# Who we are

QuantStack - a small Open Source software consultancy in Paris

<img src="src/us.png" style="margin: 0 auto;" />

In [4]:
#include <iostream>

std::cout << "Hello derse19!" << std::endl;

Hello derse19!


# N-D arrays are everywhere

- ... except for C++
- Basic building block of data science
- Images, Videos, Music, Point Clouds – all are expressed as N-dimensional data

<img src="src/xtensor.png" width="500px" />

- n-dimensional array computing library
- Support for lazy computation, broadcasting 
- C++14
- API close to NumPy
- Online demo: https://mybinder.org/v2/gh/QuantStack/xtensor/stable?filepath=notebooks/xtensor.ipynb

In [5]:
#include <xtensor/xio.hpp>
#include <xtensor/xarray.hpp>

xt::xarray<double> a = {{1,2,3}, {4,5,6}};

std::cout << a << std::endl;

{{ 1.,  2.,  3.},
 { 4.,  5.,  6.}}


### STL interface

In [6]:
for (auto& el : a)
{
    std::cout << el << std::endl;
}

1
2
3
4
5
6


In [7]:
a(0, 1) = 10;

std::cout << a << std::endl;

{{  1.,  10.,   3.},
 {  4.,   5.,   6.}}


In [8]:
std::sort(a.begin(), a.end());

std::cout << a << std::endl;

{{  1.,   3.,   4.},
 {  5.,   6.,  10.}}


In [9]:
std::cout << xt::transpose(a) << std::endl;

{{  1.,   5.},
 {  3.,   6.},
 {  4.,  10.}}


In [10]:
a.reshape({1, 1, 6, 1});

std::cout << a << std::endl;

{{{{  1.},
   {  3.},
   {  4.},
   {  5.},
   {  6.},
   { 10.}}}}


In [11]:
a.reshape({2, 3});

### Functions

In [12]:
auto xfunc = sin(a) * 2;

std::cout << xfunc << std::endl;

{{ 1.682942,  0.28224 , -1.513605},
 {-1.917849, -0.558831, -1.088042}}


### Expression Templates

In [13]:
template <class T>
void print_type(T) { std::cout << __PRETTY_FUNCTION__ << std::endl; }

In [14]:
print_type(xfunc);

void print_type(T) [T = xt::xfunction<xt::detail::multiplies, xt::xfunction<xt::math::sin_fun, const xt::xarray_container<xt::uvector<double, std::allocator<double> >, xt::layout_type::row_major, xt::svector<unsigned long, 4, std::allocator<unsigned long>, true>, xt::xtensor_expression_tag> &>, xt::xscalar<int> >]


In [15]:
auto xfunc_el = xfunc(0, 1);
std::cout << xfunc_el << std::endl;

0.28224


In [16]:
xt::xarray<double> xfunc_result = xfunc;

std::cout << xfunc_result << std::endl;

{{ 1.682942,  0.28224 , -1.513605},
 {-1.917849, -0.558831, -1.088042}}


# Broadcasting

- Automagically extend operands to match shape
- Broadcasting rules copied from NumPy

<img src="src/broadcasting.png" />

In [17]:
std::cout << a << std::endl;

{{  1.,   3.,   4.},
 {  5.,   6.,  10.}}


In [19]:
xt::xarray<double> brcast = {-10, +1, -10};
std::cout << brcast << std::endl;

input_line_29:2:21: error: redefinition of 'brcast'
 xt::xarray<double> brcast = {-10, +1, -10};
                    ^
input_line_27:3:20: note: previous definition is here
xt::xarray<double> brcast = {-10, +1, -10};
                   ^


Interpreter Error: 

In [18]:
std::cout << a * brcast << std::endl;

{{ -10.,    3.,  -40.},
 { -50.,    6., -100.}}


# Memory Layout

- Freely choose row, column-major or custom strides

In [41]:
xt::xarray<double, xt::layout_type::column_major> a_cm = {{1,2,3}, {4,5,6}};

std::cout << a_cm << std::endl;

{{ 1.,  2.,  3.},
 { 4.,  5.,  6.}}


# Container optimizations

In [42]:
// uses a std::array for shape & strides
#include <xtensor/xtensor.hpp>

xt::xtensor<double, 2> b = {{1, 2, 3}, {4, 5, 6}};
std::cout << b << std::endl;

{{ 1.,  2.,  3.},
 { 4.,  5.,  6.}}


In [43]:
// uses a fixed size container for data (std::array)
// strides computed constexpr at compile time (no storage)
#include <xtensor/xfixed.hpp>

xt::xtensor_fixed<double, xt::xshape<2, 3>> c = {{1,2,3}, {4,5,6}};
std::cout << c << std::endl;

{{ 1.,  2.,  3.},
 { 4.,  5.,  6.}}


# Views

- powerful views to reference sub-arrays
- compute shapes statically if possible

<img src="src/views1.png" />

<img src="src/views2.png" />

In [20]:
#include <xtensor/xview.hpp>
#include <xtensor/xio.hpp>
using namespace xt::placeholders;

std::cout << a << std::endl;

{{  1.,   3.,   4.},
 {  5.,   6.,  10.}}


In [21]:
std::cout << xt::view(a, 1, xt::all()) << std::endl;

{  5.,   6.,  10.}


In [22]:
// numpy equivalent: a[:, ::2]
std::cout << xt::view(a, xt::all(), xt::range(_, _, 2)) << std::endl;

{{  1.,   4.},
 {  5.,  10.}}


In [23]:
xt::view(a, xt::all(), xt::range(_, _, 2)) *= 100;

std::cout << a << std::endl;

{{  100.,     3.,   400.},
 {  500.,     6.,  1000.}}


In [24]:
std::cout << a << std::endl;

{{  100.,     3.,   400.},
 {  500.,     6.,  1000.}}


In [25]:
std::cout << xt::view(a, xt::keep(0, 0, 0) , xt::all()) << std::endl;

{{ 100.,    3.,  400.},
 { 100.,    3.,  400.},
 { 100.,    3.,  400.}}


# Adapting a 1D container

In [26]:
#include <vector>
#include <xtensor/xadapt.hpp>

std::vector<double> vdata = {1, 2, 3, 4, 5, 6, 7, 8, 9};
auto adapted_vdata = xt::adapt(vdata, {3, 3});

In [27]:
std::cout << adapted_vdata << std::endl;

{{ 1.,  2.,  3.},
 { 4.,  5.,  6.},
 { 7.,  8.,  9.}}


In [28]:
adapted_vdata(1, 2)

6

In [29]:
#include <xtensor/xmath.hpp>

std::cout << sum(adapted_vdata) << std::endl;

 45.


In [30]:
std::cout << sum(adapted_vdata, {1}) << std::endl;

{  6.,  15.,  24.}


# NumPy to xtensor Cheatsheet

<img src="src/cheatsheet.png" style="float: right" width="700px"></img>

Many functions that were not discussed:

- More reducers: `mean`, `prod` ...
- Accumulators: `cumsum`, `cumprod` ...
- Random numbers, generators
- Sorting, filtering
- Optional (masked) values
- NPY, CSV file format support

https://xtensor.readthedocs.io/en/latest/numpy.html

# xtensor

- Modular sub-packages:
    - xsimd: for SIMD vectorization support
      (batch classes, hi-speed implementations of trigo, exp, ... functions) 
    - Intel TBB: parallelization
- On top of xtensor:
    - xtensor-blas: bindings to BLAS & LAPACK – matrix multiplication, inverse, svd, ...
    - xtensor-io: image, sound, NPZ, HDF5, (soon video support)
    - xtensor-fftw, xtensor-interpolate and more

# xtensor as a building block

<img src="src/pyjlr.png" width="500px" />

- xtensor-python / julia / R
- Seamlessly use arrays from each language
- Better performance for dynamic languages

# Rayshading with xtensor

<img src="src/xtensor_langs.png" />

# Benchmarks

- Standalone C++: 0.016s
- Python + NumPy: 14.22s
- Python + xtensor: 0.022s
- Julia: 0.014s
- Julia + xtensor: 0.014s
- R: 9.905s
- R + xtensor: 0.013s

# Thirdparty packages using xtensor

- More and more packages adopting xtensor

- C++: xtensor-fftw, xtensor-interpolate, tinydnn
- Python: z5 (data format), cppcolormap
- R: rray (broadcasting n-dim matrices for R!)

### Binding a Python / NumPy container

```c++
double sum_of_sines(xt::pyarray<double>& m)
{
    auto sines = xt::sin(m);  // sines does not actually hold values.
    return std::accumulate(sines.begin(), sines.end(), 0.0);
}

PYBIND11_MODULE(xtensor_python_test, m)
{
    xt::import_numpy();
    m.doc() = "Test module for xtensor python bindings";

    m.def("sum_of_sines", sum_of_sines, "Sum the sines of the input values");
}
```

# Thanks

Check out https://github.com/QuantStack/xtensor

Follow on Twitter:

- @JohanMabille
- @wuoulf
- @QuantStack


# xtensor vs Blitz++

- No runtime dimension container in Blitz (xarray)
- Blitz++ unmaintained/dead
- Legacy code, already broken with new compilers

# xtensor vs Eigen3

- n-Dimensions from the start (Eigen has Tensor Module in /unsupported)
- modularity vs monolithical Eigen
- API subjectively "nicer", modern C++ features
- extensibility (Python/Julia/R)